# Augmented Dickey-Fuller Test

In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import math
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
qb = QuantBook()

In [ ]:
def stats(symbols):
    start_time = datetime(2018, 1, 1)
    end_time = datetime(2019, 1, 1)
    df = qb.History(symbols, start=start_time, end=end_time, resolution=Resolution.Daily)
    dg = df["close"].unstack(level=0)

    ticker1 = str(symbols[0])
    ticker2 = str(symbols[1])

    Y = dg[ticker1].apply(lambda x: math.log(x))
    X = dg[ticker2].apply(lambda x: math.log(x))

    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    sigma = math.sqrt(results.mse_resid)
    slope = results.params[1]
    intercept = results.params[0]
    res = results.resid
    zscore = res/sigma
    adf = adfuller(res)

    return [adf, zscore, slope]

In [ ]:
# Top 3 stocks obtained from above
tickers = ['BRKB', 'CMB', 'NB']
filter_fine = [qb.Symbol("BRKB R735QTJ8XC9X"), qb.Symbol("CMB R735QTJ8XC9X"), 
               qb.Symbol("NB R735QTJ8XC9X")]

In [ ]:
adfpairs = {}

# Top 3 stocks in the FinancialServices sector
print("Top 3 stocks")
for x in filter_fine:
    print(str(x))

# Form 6 distinct pairs
print("6 pairs with ADF")
for symbol1 in filter_fine:
    for symbol2 in filter_fine:
        if symbol1 != symbol2:
            p = stats([symbol1, symbol2])[0][1]
            print(str(symbol1) + ',' + str(symbol2) + ": " + str(p))
            adfpairs[str(symbol1) + ',' + str(symbol2)] = p

# Find the pair with the highest ADF
maxval = max(adfpairs, key = lambda k: adfpairs[k])
maxidx = maxval.find(",")
maxsymbol1 = qb.Symbol(maxval[:maxidx])
maxsymbol2 = qb.Symbol(maxval[maxidx+1:])
print("Pair with the highest ADF:" + maxsymbol1.Value + "," + maxsymbol2.Value)

# Find the pair with the lowest ADF
minval = min(adfpairs, key = lambda k: adfpairs[k])
minidx = minval.find(",")
minsymbol1 = qb.Symbol(minval[:minidx])
minsymbol2 = qb.Symbol(minval[minidx+1:])
print("Pair with the lowest ADF:" + minsymbol1.Value + "," + minsymbol2.Value)